In [ ]:
#!pip install pygame

In [ ]:
#!pip install tensorflow

In [ ]:
#!pip install tensorflow_probability

In [1]:
import pygame

pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Random Moves 

In [ ]:
# import random
# import graphical, game

# def ai_callback(board, score, moves_left):
#     dir = random.randint(0, 1) == 0
#     return (random.randint(0, 7 if dir else 6), random.randint(0, 8 if dir else 9), dir)

# def transition_callback(board, move, score_delta, next_board, moves_left):
#     print(f"Score: {score_delta} , moves left: {moves_left} ")
#     #pass # This can be used to monitor outcomes of moves

# def end_of_game_callback(boards, scores, moves, final_score):
#     return False # True = play another, False = Done


# if __name__ == '__main__':
#     speedup = 1.0
#     g = graphical.Game(ai_callback, transition_callback, end_of_game_callback, speedup)
#     g.run()


# Creating the Action mapping dictionary 

In [ ]:
cols = [0,1,2,3,4,5,6,7]

rows = [0,1,2,3,4,5,6,7,8,9]

dec = [0,1]

actions = []


for i in cols:
    for j in rows:
        for k in dec:
            
            # dont add furtherest right coloumn with swapping horizontal , as invalid 
            #if i==7 and k==1:
            #    pass
            
            # if top row, dont add the swapping vertical choice
            #if j==0 and k==0;
            #    pass
            
            #else:
            actions.append( (i,j,k) )
        
        
print(len(actions))

In [ ]:
print(actions)

### Convert to dictionary 

In [ ]:
# set up keys
keys = [i for i in range(160)]

print(keys)

In [ ]:
dictOfActions = dict(zip(keys, actions))

In [ ]:
print(dictOfActions)

In [ ]:
print(dictOfActions[5])

# My Attempt


In [ ]:
import numpy as np

In [ ]:
# preprocess function for board 

def preprocess_string_int(input_string):
    
    """This will firstly convert the string representation into an interger representation. Then it will convert the interger
    representation of the board state into a tensor, to be evaluated in the neural net. """
    
    # removing the \n as they are really only used to make the text output of board state more readible 
    new_string = input_string.replace('\n', "")
    
    #stores ints representation of string
    inter = [] 
    
    for i in new_string:
        
        # a  = 1
        if i == 'a':
            inter.append(1)
        
        # b  = 2
        if i == 'b':
            inter.append(2)
            
        # c  = 3
        if i == 'c':
            inter.append(3)   
            
        # d  = 4
        if i == 'd':
            inter.append(4) 
            
        # '#' = 5
        if i == '#':
            inter.append(5) 
       
    
    # finally convert to tensor for neural nets
    inter_tf = tf.convert_to_tensor(inter)
    
    inter_tf = tf.expand_dims(inter_tf, 0)
   
    
    return inter_tf

In [ ]:
test = "abcd\nbcda#\nabcda"

In [ ]:
y = preprocess_string_int(test)

print(y)

print(y.shape)

print(type(y))

# Hopefully smart-ish

# Set up of Neural Network

In [ ]:
# set up of AC model
num_inputs = 80      # board state, theres 80 options 
num_actions = 160    # allowing all actions
num_hidden_1 = 1024  # first hidden layer
num_hidden_2 = 512   # second hidden layer 


inputs = layers.Input(shape=(num_inputs,))

#first hidden layer
common_1 = layers.Dense(num_hidden_1, activation="relu")(inputs)

#second hidden layer
common_2 = layers.Dense(num_hidden_2, activation="relu")(common_1)

# choosing action 
action = layers.Dense(num_actions, activation="softmax")(common_2)
# outputting critic score  
critic = layers.Dense(1)(common_2)

model = keras.Model(inputs=inputs, outputs=[action, critic])

#  Random Choices from AC NN 

In [ ]:
# # perfectly fine random 


import random
import graphical, game

def ai_callback(board, score, moves_left):
    
    # pre-process board 
    integer_board = preprocess_string_int(board)
    
    print(integer_board)
    print(len(integer_board))
    
    #################################################
    
    # now actor critic to make decision of choice 

    action_probs, critic_val = model(integer_board)
    
    print(action_probs)
    
    # Sample action from action probability distribution, higher probababilites more chance of getting selected
    action = np.random.choice(num_actions, p=np.squeeze(action_probs))
         
    return dictOfActions[action]
            
    
def transition_callback(board, move, score_delta, next_board, moves_left):
    print(f"Score: {score_delta} , moves left: {moves_left} ")
    #pass # This can be used to monitor outcomes of moves

def end_of_game_callback(boards, scores, moves, final_score):
    return False # True = play another, False = Done


if __name__ == '__main__':
    speedup = 1.0
    g = graphical.Game(ai_callback, transition_callback, end_of_game_callback, speedup)
    g.run()


In [ ]:
import random
import graphical, game

def ai_callback(board, score, moves_left):
    
    # pre-process board 
    integer_board = preprocess_string_int(board)
    
    print(integer_board)
    print(len(integer_board))
    
    #################################################
    
    # now actor critic to make decision of choice 
    
    # training
    gamma = 0.99  # Discount factor for past rewards
    max_steps_per_episode = 25
    
    optimizer = keras.optimizers.Adam(learning_rate=0.01)
    huber_loss = keras.losses.Huber()
    action_probs_history = []
    critic_value_history = []
    rewards_history = []
    running_reward = 0
    episode_count = 0
    
    
    #while True:  # Run until solved
    state = integer_board
    episode_reward = 0
    
    
    with tf.GradientTape() as tape:
        for timestep in range(1, max_steps_per_episode):
            # env.render(); Adding this line would show the attempts
            # of the agent in a pop up window.

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # Predict action probabilities and estimated future rewards
            # from environment state
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0, 0])

            # Sample action from action probability distribution
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            # Apply the sampled action in our environment
            state, reward, done, _ = env.step(action)
            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # Calculating loss values to update our network
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up recieving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # Backpropagation
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Clear the loss and reward history
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))
       

    dir = random.randint(0, 1) == 0
    return (random.randint(0, 7 if dir else 6), random.randint(0, 8 if dir else 9), dir)

def transition_callback(board, move, score_delta, next_board, moves_left):
    print(f"Score: {score_delta} , moves left: {moves_left} ")
    #pass # This can be used to monitor outcomes of moves

def end_of_game_callback(boards, scores, moves, final_score):
    return False # True = play another, False = Done


if __name__ == '__main__':
    speedup = 1.0
    g = graphical.Game(ai_callback, transition_callback, end_of_game_callback, speedup)
    g.run()


https://keras.io/examples/rl/actor_critic_cartpole/